# 1 Implementing your own Shazam
Shazam is a great application that can tell you the title of a song by listening to a short sample. For this first task, we will implement a simplified copy of this app by dealing with hashing algorithms.

## 1.1 Getting your data!
We will test this part with some query songs which we would like to know the title of (see the file `queries.zip` in the `files` branch of the current repository). To do so we will first need to convert the tracks in the dataset from mp3 format to wav format.

In [1]:
import numpy as np      
import pandas as pd
import matplotlib.pyplot as plt 
import csv
pd.set_option("display.max_columns", None)

In [15]:
import sys
import scipy.io.wavfile 
import subprocess
import librosa
import librosa.display
import IPython.display as ipd
from sympy import nextprime
from pathlib import Path, PurePath   
from tqdm.notebook import tqdm

In [9]:
N_TRACKS = 1413
HOP_SIZE = 512
OFFSET = 1.0

In [10]:
# REMARK: the files "queries" could be saved in a differen path in your local machine
# to run this cell change the path accordingly
data_folder = Path("mp3s-32k")
test_data_folder = Path('queries')

new_tracks = test_data_folder.glob('*.wav')

mp3_tracks = data_folder.glob("*/*/*.mp3")
tracks = data_folder.glob("*/*/*.wav")

In [16]:
def convert_mp3_to_wav(audio:str) -> str:  
    """Convert an input MP3 audio track into a WAV file.

    Args:
        audio (str): An input audio track.

    Returns:
        [str]: WAV filename.
    """
    if audio[-3:] == "mp3":
        wav_audio = audio[:-3] + "wav"
        if not Path(wav_audio).exists():
                subprocess.check_output(f"ffmpeg -i {audio} {wav_audio}", shell=True)
        return wav_audio
    
    return audio


In [18]:
for track in tqdm(mp3_tracks, total=N_TRACKS):
    convert_mp3_to_wav(str(track))

  0%|          | 0/1413 [00:00<?, ?it/s]

CalledProcessError: Command 'ffmpeg -i mp3s-32k/metallica/Metallica/01-Enter_Sandman.mp3 mp3s-32k/metallica/Metallica/01-Enter_Sandman.wav' returned non-zero exit status 126.

## 1.2 Fingerprint hashing

We now want to create a representation of our audio signal that allows us to characterize it with respect to its peaks. Once this process is complete, we can adopt a hashing function to get a fingerprint of each song.

**Utility functions**

In [ ]:
def plot_spectrogram_and_peaks(track:np.ndarray, sr:int, peaks:np.ndarray, onset_env:np.ndarray) -> None:
    """Plots the spectrogram and peaks 

    Args:
        track (np.ndarray): A track.
        sr (int): Aampling rate.
        peaks (np.ndarray): Indices of peaks in the track.
        onset_env (np.ndarray): Vector containing the onset strength envelope.
    """
    times = librosa.frames_to_time(np.arange(len(onset_env)),
                            sr=sr, hop_length=HOP_SIZE)

    plt.figure()
    ax = plt.subplot(2, 1, 2)
    D = librosa.stft(track)
    librosa.display.specshow(librosa.amplitude_to_db(np.abs(D), ref=np.max),
                            y_axis='log', x_axis='time')
    plt.subplot(2, 1, 1, sharex=ax)
    plt.plot(times, onset_env, alpha=0.8, label='Onset strength')
    plt.vlines(times[peaks], 0,
            onset_env.max(), color='r', alpha=0.8,
            label='Selected peaks')
    plt.legend(frameon=True, framealpha=0.8)
    plt.axis('tight')
    plt.tight_layout()
    plt.show()
    

def load_audio_peaks(audio, offset, duration, hop_size):
    """Load the tracks and peaks of an audi
    Args:
        audio (string, int, pathlib.Path or file-like object): [description]
        offset (float): start reading after this time (in seconds)
        duration (float): only load up to this much audio (in seconds)
        hop_size (int): the hop_length

    Returns:
        tuple: Returns the audio time series (track) and sampling rate (sr), a vector containing the onset strength envelope
        (onset_env), and the indices of peaks in track (peaks).
    """
    try:
        track, sr = librosa.load(audio, offset=offset, duration=duration)
        onset_env = librosa.onset.onset_strength(track, sr=sr, hop_length=hop_size)
        peaks = librosa.util.peak_pick(onset_env, 10, 10, 10, 10, 0.5, 0.5)
    except Error as e:
        print('An error occurred processing ', str(audio))
        print(e)

    return track, sr, onset_env, peaks

**Preprocessing**

In [ ]:
# To be tuned
DURATION = 30 
THRESHOLD = 0

# audio files
U = []
audio_peaks = []
for idx, audio in enumerate(tqdm(tracks, total=N_TRACKS)):
#     if idx >= 10:
#         break
    track, sr, onset_env, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE)
#     print('Track is: ', track) 
#     print('sr is: ', sr)
#     print('onset_env is: ', onset_env)
#     print('peaks is: ', peaks)
#     print('audio is: ', audio)
    U += list(peaks)
    audio_peaks.append((audio, set(peaks)))
#   plot_spectrogram_and_peaks(track, sr, peaks, onset_env)
        
U = set(U)

# load new audios
new_audio_peaks = []
for idx, audio in enumerate(tqdm(new_tracks, total=4)):
    track, sr, onset_env, peaks = load_audio_peaks(audio, OFFSET, DURATION, HOP_SIZE)
    new_audio_peaks.append((audio, set(peaks)))


new_audio_peaks

### MinHash

We will now implement a **minhash function** from scratch. After that we will read the dataset sequentially and add it to our **MinHash**. The aim is to make our minhash function to map the same song to the same bins. Then we can also define a threshold to control the accuracy of these matches. 

In [ ]:
# REMARK: MinHash is an approximation of  the Jaccard Similarity
def jaccard_similarity(list1, list2):
    """
    :param list1: first set
    :param list2: second set
    :return: the jaccard similarity between the two sets (it's between 0 and 1)
    This function is the function that has to be estimated using the minhashing principle.
    """
    intersection = len(list(set(list1).intersection(list2)))
    union = (len(set(list1)) + len(set(list2))) - intersection
    return float(intersection) / union



In [ ]:
# U = list(range(0, 200))
def generate_minhash_fns(u, n):
    """
    :param u: total number of peaks (a, b and c should be less than u)
    :param n: number of hash functions to generate
    :return: an array of hashfunctions (in the form of tuples)
    """
    minhash_functions = []
    c = nextprime(u)
    for _ in range(n):
        a = np.random.randint(u)
        b = np.random.randint(u)
        minhash_functions.append(np.array([a, b, c]))
    return minhash_functions

u = len(U)
n_minhash_fns = 20
#minhash_fns = [np.random.randint(u, size=3) for _ in range(n_minhash_fns)]
minhash_fns = generate_minhash_fns(u, n_minhash_fns)
print('aaa ', u)


def perm(x, a, b, c):
    """
    Simulates permutation using modulus 
    :param x: int
    :param a: param (int)
    :param b: param (int)
    :param c: param(int)
    :return: 
    """

    return (a * x + b) % c


def minhash(s, a, b, c):
    """
    :param s: set
    :param a: param
    :param b: param
    :param c: param
    :return: the min hash
    """    
    h = [perm(e, a, b, c) for e in s]
    mh = min(h)
    return mh


def minhash_sign(s):
    """
    :param s: set
    :return: minhash signature with different minhashes
    """
    minhashes = []
    for a, b, c in minhash_fns:
        minhashes.append(minhash(s, a, b, c))
    return minhashes


def minhash_sim(a, b):
    """
    :param a: first set
    :param b: second set
    :return: estimated Jaccard similarity using minhashing
    """

    sign_a = minhash_sign(a)
    sign_b = minhash_sign(b)
    l = len(sign_a)
    t = 0
    for a, b in zip(sign_a, sign_b):
        if a == b:
            t += 1
    return t/l

### Testing the results

In [ ]:
jaccard_similarity(new_audio_peaks[0][1], audio_peaks[0][1]) 

thresholds = [0.6, 0.7, 0.9]
for threshold in thresholds:
    results = []
    for new_audio in tqdm(new_audio_peaks):
        max_similarity = -1
        most_similar = ''
        for audio in audio_peaks:
            sim = minhash_sim(new_audio[1], audio[1])
            if sim > max_similarity and sim <= threshold:
                max_similarity = sim
                most_similar = audio[0]
        results.append((new_audio[0], most_similar, max_similarity))
        print('Results for threshold ', threshold, ' is: ', results)
            
    

results = []
for new_audio in tqdm(new_audio_peaks):
    max_similarity = -1
    most_similar = ''
    for audio in audio_peaks:
        sim = minhash_sim(new_audio[1], audio[1])
        if sim > max_similarity:
            max_similarity = sim
            most_similar = audio[0]
    results.append((new_audio[0], most_similar, max_similarity))
            
        
results

print('a')

# 2 Grouping songs together!

The final goal of this part is to group songs into similar genres (without using the feature genre in our K-means anaylsis)

## 2.1  Getting the data and Data Wrangling

We start by reading the datasets we where given.

In [ ]:
# REMARK: the execution of this cell can take a few seconds

tracks = pd.read_csv("tracks.csv")
echonest = pd.read_csv("echonest.csv")
features = pd.read_csv("features.csv")

**Traks dataset**

We see that the datasets have many features (e.g. *tracks.csv* has 106574 rows and 53 columns).

In [ ]:
print(tracks.shape)
tracks.head()

**Echonest dataset**

In [ ]:
print(echonest.shape)
echonest.head()

**Features dataset**

In [ ]:
print(features.shape)
features.head()

### Cleaning of the datasets

Since the `track_id` seems one of the most important features we check wheter there are missing or duplicate values in these columns

In [ ]:
print(tracks["track_id"].isnull().any())
print(features["track_id"].isnull().any())
print(echonest["track_id"].isnull().any())

In [ ]:
if tracks.track_id.nunique() == len(tracks["track_id"]):
    print("No duplicate track ids")
else:
    print("There are duplicate track ids")

By exploring the datasets, we've seen that some columns have many NaN values and strange entries (e.g. in `album_information`). For this reason we start filling and cleaning  the datasets. The filling part is done following some rules of thumb (see the `fill` function below):
- replace a missing numeric values (as NaN) with the mean of all the numeric values of the same kind
- replace all the NaN strings with a blanc space 

The cleaning part is done by replacing some unpleasant html tags-type-of-strings. For example in the `album_information` column of the `tracks` datasets there are strings like `<p>` we wolud like to eliminate (see the `clean` function below)

In [ ]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [ ]:
def fill(df):
    """
    Input: df, a dataframe
    Output: None
    Remark: the function performs some trasformations on the columns of the dataframe given in input, in particular:
                - if there is a missing numeric value (as NaN) it fills it with an average value
                - if there is a string wich is NaN it replaces it with a blanc space 
    """
    for i in df.columns:
        if df[i].isnull().any() == True:
            if is_numeric_dtype(df[i]) == True:
                df[i] = df[i].fillna(df[i].mean())
            elif is_string_dtype(df[i]) == True:
                    df[i] = df[i].fillna("")

In [ ]:
def clean(df):
    """
    Input: df, a dataframe
    Output: None
    Remark: the function replaces every html substring in the dataframe's columns with a blanc space
    """
    for i in df.columns:
            if is_string_dtype(df[i]) == True:
                    df[i] = df[i].str.replace(r'<[^<>]*>', '', regex=True)

In [ ]:
# copy the datasets in order to keep the original files unchanged
cleaned_tracks = tracks.copy()
cleaned_echonest = echonest.copy()
cleaned_features = features.copy()

In [ ]:
fill(cleaned_tracks)

In [ ]:
clean(cleaned_tracks)

**Remark .** In 2.2 we will apply PCA and to do so we will need to process and transform the datasets (standardization). In the this phase we will rescale the numeric columns. Since `album_id` and `artist_id` are a fixed number that corresponds to an album and an artist we don't want to change them in the process. This is why we will momentarely drop these features.  

In [ ]:
cleaned_tracks = cleaned_tracks.drop(["album_id", "artist_id"], axis = 1)

In [ ]:
print(cleaned_tracks.shape)
cleaned_tracks.head()

We now will fill and clean the `echonest` dataset.

In [ ]:
fill(cleaned_echonest)

In [ ]:
clean(cleaned_echonest)

In [ ]:
print(cleaned_echonest.shape)
cleaned_echonest.head()

**Remark .** We've decided to drop some columns to reduce the size of the `echonest` dataset. For example we believe that in our analysis `metadata_album_date`, `metadata_album_name` ecc... are not relevant, in particular those features are most of the time empty.

In [ ]:
cleaned_echonest = cleaned_echonest.drop(["metadata_album_date", "metadata_album_name"], axis = 1)

We now will fill and clean the `features` dataset.

In [ ]:
fill(cleaned_features)

In [ ]:
clean(cleaned_features)

In [ ]:
cleaned_features.head()

### Merge of the datasets

As we saw with the exploratory data analysis, the `echonest` dataset has 13.129 unique `track_id`s, while the `features` and `tracks` dataset have 106.574 unique `tracks_id`. The datasets are related to each other:
- `echonest` has some information about the audio features, metadata, ranks ecc...
- `features` contains many numerical entries that should uniquely characterize the tracks.
- `tracks` contains many informations on the album, the artist and the track.

Since the datasets have `track_id` in common, we decided to use it as a key to join the datasets. In particular we will merge the datasets w.r.t. `track_id` keeping the `echonest` dataset's rows ($\sim 13000$ rows). We than fill and clean the dataset as it was done before.

In [ ]:
# merge tracks and echonest w.r.t. track_id
first_merge = cleaned_tracks.merge(cleaned_features, on = "track_id")
dataset = first_merge.merge(cleaned_echonest, on = "track_id")

In [ ]:
fill(dataset)

In [ ]:
clean(dataset)

In [ ]:
print(dataset.shape)
dataset.head()

## 2.2 Choose your features (variables)!
We have plenty of features to work with (> 800 columns). So, we will need to find a way to reduce the dimensionality. We will use **PCA** (Principal Component Analysis) for **Dimensionality Reduction**. 

**Disclamair**. We've gained many insight from https://towardsdatascience.com/principal-component-analysis-for-dimensionality-reduction-115a3d157bad. 

In [ ]:
from sklearn.decomposition import PCA
from sklearn import preprocessing

Since we would like to apply PCA, first we need to select all the numeric features frome the dataset(s).

In [ ]:
processed_dataset = dataset.select_dtypes([np.number])

In [ ]:
print(processed_dataset.shape)
processed_dataset.head()

Now we "standardize" the features by removing the mean and scaling to unit variance. This transformations can be done using `sklearn preprocessing`, in particular using `sklearn.preprocessing.StandardScaler`.

**Remark .** In this preprocessing part we will ignore some features (e.g. `track_id`). Indeed it isn't good to change some fixed features that are related to the ordeting of the songs or the id of the tracks and the albums. Recall that we have already ignored `album_id` and `artist_id` in 2.1. At the end of this processing part we will restore some of the columns we have previously ignored.

In [ ]:
# this dataset can be also be used to carry out some other dimensionality reduction
# we will use it for the PCA

df = processed_dataset

In [ ]:
scaler = preprocessing.StandardScaler()

# processed dataset without trak_id as a feature
temp_df = pd.DataFrame(scaler.fit_transform(df[df.columns[1:]].values), columns = df.columns[1:])

In [ ]:
#print(final_df.shape)
temp_df.head()

**Remark.** In this temporary dataframe there are many columns that we will never use in our future analysis, that is why we drop them. Moreover, by reducing the number of columns we will then have a more accurate clustering (cfr. 2.4) while reducing the number of components in the PCA. If the following columns where not to be dropped the neede components to retain more then 70% of the variance would be 75 instead of 70 and the successive clustering would be less accurate.

In [ ]:
temp_df = temp_df.drop(["metadata_artist_longitude", 
                            "ranks_artist_discovery_rank",
                            "ranks_artist_familiarity_rank",
                            "ranks_artist_hotttnesss_rank",
                            "ranks_song_currency_rank",
                            "ranks_song_hotttnesss_rank",
                            "artist_comments","artist_favorites",
                            "artist_latitude","track_number"],
                           axis = 1)

In [ ]:
print(temp_df.shape)
temp_df.head()

### PCA with 70 components in order to have > 70% of the total variance 
We decided to keep this number of features because it is both one order of magnitude less than the original number of features and a big enough to keep a good percentage of the original variance.

In [ ]:
# dimensionality reduction
pca = PCA(n_components=70)
df_pca = pca.fit_transform(temp_df)

# retained total variance
print(sum(pca.explained_variance_ratio_))

In [ ]:
# convert the result of PCA into a pandas dataframe
# this reduced dataset misses some important features, e.g. track's duration or language
# some of those features will be restored in the final_reduced_dataset below
final_df_pca = pd.DataFrame(df_pca, columns = ['Feature_%i' % i for i in range(70)])

In [ ]:
print(final_df_pca.shape)
final_df_pca.head()

### Interpretation of the results
By plotting the contribution of each components to the retained total variance we see that the first features contribute the most to the retained variance. We se, for example, that `Feature_0` contributes to the 10.6% of the total variance and uccessive features then contribute lesser and lesser.

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.plot(pca.explained_variance_ratio_, marker=".", markersize=8)
plt.ylabel('Retained Variance')
plt.xlabel('Features')
plt.show()

In [ ]:
pca.explained_variance_ratio_

The following plot shows that by increasing the number of retained features the total retained variance increases. It is interesting to note that by keeping just 30 features we would have retained more than the 50% of the total variance. This suggests that many initial features where redundant.

In [ ]:
var = np.cumsum(np.round(pca.explained_variance_ratio_,decimals = 3)*100)

In [ ]:
var

In [ ]:
fig = plt.figure(figsize=(12,6))
plt.plot(var, marker=".", markersize=8)
plt.ylabel('Retained Variance')
plt.xlabel('number of Features')
plt.show()

### Complete the reduced dataset with useful informations
We add back some important features, such as `track_duration` and `track_language_code` which we believe will be useful for further analysis

In [ ]:
# this is the final reduced dataset we will use for the K-means 
final_df_pca = pd.concat([
                    dataset.loc[:, [
                            'track_id',
                            'track_genre_top', 
                            'track_duration',
                            'audio_features_tempo',
                            'track_language_code',
                            'metadata_artist_location',
                            'audio_features_acousticness',
                            'audio_features_danceability',
                            'audio_features_energy',
                            'audio_features_instrumentalness',
                            'audio_features_liveness',
                            'audio_features_speechiness'
                            ]
                    ],
                    final_df_pca],
                    axis = 1
                    )

In [ ]:
print(final_df_pca.shape)
final_df_pca.head()

## 2.3 Clustering

We will now implement the *K-means clustering algorithm* (not ++: random initialization) from scratch. Then we will find an optimal number of clusters, running the algorithm on the data from the previous dimensionality reduction. At the end we will use the already implemented version of k-means++ (from the `scikit-learn` library) to compare the results.

In [ ]:
# for kmeans_utils the module "kneed" needs to be installed
import kmeans_utils as kmu
from sklearn.cluster import KMeans

In [ ]:
del tracks; del echonest; del features 

In [ ]:
df_pca = final_df_pca.iloc[:, 12:]
df_pca.head()

In [ ]:
# REMARK: the execution of this cell can take a few seconds

k, score_results = kmu.get_best_k(kmu.MyKM, df_pca, max_k=8, seed=42)

In [ ]:
print('Best k is:', k)

### Comparison between our Kmeans and Scikit-learn Kmeans++
**Initialization:** In our kmeans (MyKM) the number of clusters is provided in the fit method and not in the instance creation (i.e.: not imploemented in \__init\__)

In [ ]:
# Initialization
my_km = kmu.MyKM(n_clusters=4, seed=42)
km = KMeans(n_clusters=4, random_state=42)

**Fitting:** Scikit-learn kmeans is faster than ours implementation. It takes less than 1s to train 10 models whereas our model takes about 10s for a fit.

In [ ]:
my_km.fit(df_pca, info=True)

In [ ]:
import time
t0 = time.time()
print(km.fit(df_pca), 'Computing time:', time.time() - t0)

**Results:** The labels are different for the two algorithms. In order to compare them we will compare the value of inertia.
Our result seems to bu much better , in fact its about 40% than scikit-learn's kmeans:

In [ ]:
print('my_km inertia:', my_km.inertia_)
print('km inertia:', km.inertia_)

Saving the model for further analysis

In [ ]:
kmu.save_model(my_km)

## 2.4 Analysing the results



Load the model:

In [ ]:
my_km = kmu.load_model()

Adding the computed labels to the dataset

In [ ]:
final_df_pca['labels'] = my_km.labels_

Check the dataset head

In [ ]:
final_df_pca.head()

**1. Language**

In [ ]:
kmu.get_crosstab('track_language_code', final_df_pca)

**2. Track duration**

In [ ]:
kmu.get_crosstab('track_duration', final_df_pca)

**3. Tempo** 

In [ ]:
kmu.get_crosstab('audio_features_tempo', final_df_pca)

**4. Acousticness**

In [ ]:
kmu.get_crosstab('audio_features_acousticness', final_df_pca)

**5. Danceability**

In [ ]:
kmu.get_crosstab('audio_features_danceability', final_df_pca)

**6. Energy**

In [ ]:
kmu.get_crosstab('audio_features_energy', final_df_pca)

**7. Instrumentalness**

In [ ]:
kmu.get_crosstab('audio_features_instrumentalness', final_df_pca)

**8. Liveness**

In [ ]:
kmu.get_crosstab('audio_features_liveness', final_df_pca)

**9. Speechiness**

In [ ]:
kmu.get_crosstab('audio_features_speechiness', final_df_pca)

### Genre

In [ ]:
kmu.get_crosstab('track_genre_top', final_df_pca)

###  KMeans++ Analysis
Replacing the  MyKM labels with the ones provided by scikit-learn KMeans++

In [ ]:
best_k, score_results = kmu.get_best_k(KMeans, df_pca, max_k=8, random_state=42)

In [ ]:
km_opt = KMeans(n_clusters=4, random_state=42).fit(df_pca)

In [ ]:
final_df_pca['labels'] = km_opt.labels_

**1. Language**

In [ ]:
kmu.get_crosstab('track_language_code', final_df_pca)

**2. Track duration**

In [ ]:
kmu.get_crosstab('track_duration', final_df_pca)

**3. Tempo** 

In [ ]:
kmu.get_crosstab('audio_features_tempo', final_df_pca)

**4. Acousticness**

In [ ]:
kmu.get_crosstab('audio_features_acousticness', final_df_pca)

**5. Danceability**

In [ ]:
kmu.get_crosstab('audio_features_danceability', final_df_pca)

**6. Energy**

In [ ]:
kmu.get_crosstab('audio_features_energy', final_df_pca)

**7. Instrumentalness**

In [ ]:
kmu.get_crosstab('audio_features_instrumentalness', final_df_pca)

**8. Liveness**

In [ ]:
kmu.get_crosstab('audio_features_liveness', final_df_pca)

**9. Speechiness**

In [ ]:
kmu.get_crosstab('audio_features_speechiness', final_df_pca)

 **10. Genre**

In [ ]:
kmu.get_crosstab('track_genre_top', final_df_pca)

# 3. Algorithmic questions

Given a list of integers, A, and another integer s. Write an algorithm that outputs all the pairs in A that equal x.

*Example* . If $A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3]$ and $s = 4$, then the algorithm should output: $(7, -3), (-2, 6), (2, 2), (3, 1)$.

A **simple solution** is to inspect each element of the list and check if there’s another number in the array which can be added to it to give sum. The simple solution has complexity $O(n^2)$ wich can be improved (e.g. by sorting the array first and than using binary search). A **second and more efficient solution** is given using Hash Tables: 
- Consider a Hash Table H 
- For each element $A[i]$ check if $s-A[i]$ is in $H$ 

The time complexity in this case is $O(n)$. 

In [ ]:
# the following code is shorter then the simple solution
# but the complezity is O(n!)
from itertools import combinations
def algo1(A, s):
    a = combinations(A, 2)
    return list(filter(lambda x: sum(x) == s, a ))

In [ ]:
A = [7, -2, 8, 2, 6, 4, -7, 2, 1, 3, -3] 
s = 4

In [ ]:
algo1(A, s)

In [ ]:
# simple solution using an upper triangular matrix
# complexity of O(n^2)
def algo2(A,s) :
    l=[]
    for i in range (len(A)-1):
        for j in range (i + 1, len(A)):
            if (A[i] + A[j] == s):
                l.append((A[i] , A[j]))
    return(l)

In [ ]:
algo2(A,s)

In [ ]:
# efficient solution with Hash tables
# complexity O(n)
def algo3(A,s):
    count = 0
    # the set X will work as an Hash Table for the list A
    X = set(A)
    # list to keep track of all the pairs with sum s
    couples = []
    
    for item in A:
        if s - item in X:
            # min and max are used to discard duplicates
            couples.append ((min(item, s-item), max(item, s-item)))
            count+=1
    set_pairs = set(couples)
    print ("Pairs which sum up to ",s," are: ", set_pairs)


In [ ]:
algo3(A,s)